In [2]:
import numpy as np
import math
yi = 0.04255603018307094
n = 5
dates = [4/12, 10/12, 16/12, 22/12, 28/12]
cashflows = [2, 2, 2, 2, 102]
def r_zero(t):
    return 0.015 + (1 + 2* (t**2)) / (100 + 100 * (t**2))

def discountfactors(n, t_cf, v_cf, r_zero):
    discount = [0]*n
    for i in range (0, n):
        discount[i] = np.exp(-t_cf[i] * r_zero(t_cf[i]))
    return discount
 
def bond_price(n, t_cf, v_cf, r_zero):
    B = 0 #bond price
    disc = [0]*n
    for i in range (0, n):
        disc[i] = np.exp(-t_cf[i] * r_zero(t_cf[i]))
        B = B + v_cf[i] * disc[i]
    return B

def bond_price_withy(n, t_cf, v_cf, y):
    B = 0 #bond price
    disc = [0]*n
    for i in range (0, n):
        disc[i] = np.exp(-t_cf[i] * y)
        B = B + v_cf[i] * disc[i]
    return B

price = bond_price(n, dates, cashflows, r_zero)
price

102.07845573643802

In [8]:
discs = discountfactors(n, dates, cashflows, r_zero)
discs

[0.9913707806297248,
 0.9760430022280248,
 0.9589976384002267,
 0.9417995254855543,
 0.9249219004210878]

In [3]:


def D_mod(n, t_cf, v_cf, y):
    for i in range (0, n):
        res = 0
        mod = t_cf[i]*v_cf[i]*np.exp(-y * t_cf[i]) / price
        res = res + mod
    return res

Dmod = D_mod(n, dates, cashflows, yi)
Dmod

1.889909571506576

In [4]:
def Convexity(n, t_cf, v_cf, y):
    for i in range (0, n):
        res = 0
        conv = (t_cf[i]*t_cf[i])*v_cf[i]*np.exp(-y * t_cf[i]) / price
        res = res + conv
    return res

Conv = Convexity(n, dates, cashflows, yi)
Conv

4.409789000182012

In [5]:
dollar_D = Dmod * price
dollar_D

192.91905054090455

In [6]:
DV01 = dollar_D / 10000
DV01

0.019291905054090456

In [7]:
dollar_Convexity = Conv * price
dollar_Convexity

450.14445126211075

In [9]:
def BnewD(B, D, deltay):
    return B * (1 - D * deltay)

def BnewDC(B, D, deltay, C):
    return B * (1 - D * deltay + 0.5 * C * (deltay**2))


deltas = [0.0010, 0.0050, 0.01, 0.02, 0.04]
newDs = [0]*5
newDCs = [0]*5
for i in range(0, 5):
    newDs[i] = BnewD(price, Dmod, deltas[i])
    
    
#another approach   
for i in range(0, 5):
    newDCs[i] = BnewDC(price, Dmod, deltas[i], Conv) 
    
real_price = [0]*5
for i in range(0, 5):
    real_price[i] = (bond_price_withy(n, dates, cashflows, yi+deltas[i]))
    
    

In [10]:
for i in range(0, 5):
    display(np.abs(newDs[i] - real_price[i]) / real_price[i])

0.00012616627540767912

0.0005990134548211533

0.0011175802963500395

0.0019072885093665978

0.0024533027633223234

In [11]:
for i in range(0, 5):
    display(np.abs(newDCs[i] - real_price[i]) / real_price[i])

0.00012790815039456075

0.0006428865652465946

0.0012947170658479592

0.0026291673622738532

0.0054503615361830345